In [1]:
import sys

sys.path.append('modules/')
from modules import goodreads, notion


# Provide the Goodreads URL to capture the metadata for
URL = "https://www.goodreads.com/book/show/55275019-machine-learning-design-patterns?ref=nav_sb_ss_1_111"

book = await goodreads.get_book_metadata(URL)

AttributeError: 'NoneType' object has no attribute 'text'

In [9]:
book

<coroutine object get_book_metadata at 0x13f963340>

In [7]:

# Print the metadata for the book
print(f'Name: {book.name}')
print(f'Image: {book.image}')
print(f'Format: {book.book_format}')
print(f'Number of pages: {book.num_pages}')
print(f'Language: {book.language}')
print(f'ISBN: {book.isbn}')
print(f'Rating: {book.rating}')
print(f'Number of ratings: {book.rating_count}')
print(f'Number of reviews: {book.review_count}')
print(f'Authors: {book.authors}')

AttributeError: 'coroutine' object has no attribute 'name'

In [27]:
notion.post_book_from_goodreads(book)

TypeError: 'GoodReadBook' object is not subscriptable

In [8]:

from bs4 import BeautifulSoup
import requests

r = requests.get(URL)
soup = BeautifulSoup(r.text, "html.parser")

script = soup.find('script', {'type': 'application/ld+json'}).text

AttributeError: 'NoneType' object has no attribute 'text'

In [12]:
soup.find('span', {'itemprop': 'isbn'}).text
soup.find_all('a', {'class': 'authorName'})
soup.find('div', {'itemprop': "inLanguage"}).text


<!DOCTYPE html>

<html class="desktop withSiteHeaderTopFullImage">
<head prefix="og: http://ogp.me/ns# fb: http://ogp.me/ns/fb# good_reads: http://ogp.me/ns/fb/good_reads#">
<title>Machine Learning Design Patterns: Solutions to Common Challenges in Data Preparation, Model Building, and MLOps by Valliappa Lakshmanan</title>
<meta content="Machine Learning Design Patterns book. Read 21 reviews from the world's largest community for readers. The design patterns in this book capture best prac..." name="description"/>
<meta content="telephone=no" name="format-detection"/>
<link href="https://www.goodreads.com/book/show/55275019-machine-learning-design-patterns" rel="canonical"/>
<meta content="2415071772" property="fb:app_id"/>
<meta content="books.book" property="og:type"/>
<meta content="Machine Learning Design Patterns" property="og:title"/>
<meta content="The design patterns in this book capture best practices and solutions to recurring problems in machine learning. Authors Valliappa La

In [1]:
import scrapy
from scrapy.crawler import CrawlerProcess, CrawlerRunner
from multiprocessing import Process, Queue
from twisted.internet import reactor

class GoodreadsSpider(scrapy.Spider):
    name = 'goodreads'
    start_urls = ['https://www.goodreads.com/book/show/55275019-machine-learning-design-patterns?ref=nav_sb_ss_1_111']

    def parse(self, response):
        book_title = response.css('#bookTitle::text').get()
        book_author = response.css('a.authorName>span::text').get()
        book_cover = response.css('.bookCover::attr(src)').get()
        book_rating = response.css('span[itemprop=ratingValue]::text').get()
        book_isbn = response.css('div.infoBoxRowItem[itemprop=isbn]::text').get()
        book_language = response.css('div[itemprop=inLanguage]::text').get()

        print(book_title)
        print(book_author)
        print(book_cover)
        yield {
            'title': book_title,
            'author': book_author,
            'cover': book_cover,
            'rating': book_rating,
            'isbn': book_isbn,
            'language': book_language
        }


# def f(q, spider):
#     try:
#         runner = CrawlerRunner()
#         deferred = runner.crawl(spider)
#         deferred.addBoth(lambda _: reactor.stop())
#         reactor.run()
#         q.put(None)
#     except Exception as e:
#         q.put(e)
#
#     return q

# the wrapper to make it run more times
# def run_spider(spider):
#
#     queue = Queue()
#     process = Process(target=f, args=(queue, spider))
#     process.start()
#     result = queue.get()
#     process.join()
#
#     if result is not None:
#         raise result

# Run the spider
# run_spider(GoodreadsSpider)
process = CrawlerProcess()
process.crawl(GoodreadsSpider)
process.start()


2022-12-21 02:09:47 [scrapy.utils.log] INFO: Scrapy 2.7.1 started (bot: scrapybot)
2022-12-21 02:09:47 [scrapy.utils.log] INFO: Versions: lxml 4.9.2.0, libxml2 2.9.13, cssselect 1.2.0, parsel 1.7.0, w3lib 2.1.1, Twisted 22.10.0, Python 3.11.1 (v3.11.1:a7a450f84a, Dec  6 2022, 15:24:06) [Clang 13.0.0 (clang-1300.0.29.30)], pyOpenSSL 22.1.0 (OpenSSL 3.0.7 1 Nov 2022), cryptography 38.0.4, Platform macOS-13.1-arm64-arm-64bit
2022-12-21 02:09:47 [scrapy.crawler] INFO: Overridden settings:
{}
2022-12-21 02:09:47 [py.warnings] WARNING: /Users/eric/.local/share/virtualenvs/knowledge-toolkit-VJMUfH12/lib/python3.11/site-packages/scrapy/utils/request.py:231: ScrapyDeprecationWarning: '2.6' is a deprecated value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting.

It is also the default value. In other words, it is normal to get this warning if you have not defined a value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting. This is so for backward compatibility reasons, but it will chang

None
None
None


In [3]:
from requests_html import AsyncHTMLSession
from bs4 import BeautifulSoup


URL = "https://www.goodreads.com/book/show/55275019-machine-learning-design-patterns?ref=nav_sb_ss_1_111"

asession = AsyncHTMLSession()
r = await asession.get(URL)
await r.html.arender()

soup = BeautifulSoup(r.html.raw_html, "lxml")

In [4]:
from modules.utils import GoodReadBook
import json
script = soup.find('script', {'type': 'application/ld+json'}).text
book = GoodReadBook(json.loads(script), URL)


In [5]:
book.name

'Machine Learning Design Patterns: Solutions to Common Challenges in Data Preparation, Model Building, and MLOps'